In [1]:
import youtube_transcript_api
from youtube_transcript_api import YouTubeTranscriptApi
import nltk
import re
from nltk.corpus import stopwords
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize
#nltk.download('punkt_tab')
import numpy as np
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration

In [25]:
# get the link
link = "https://www.youtube.com/watch?v=GeCodRjs1Bs" 
# get the unique ID
unique_id = link.split("=")[-1]
# get the transcript of the video based on the unique ID
sub = YouTubeTranscriptApi.get_transcript(unique_id)  
# get only the "text" parts and join them together, ignoring others like "duration"
subtitle = " ".join([x['text'] for x in sub])

In [26]:
print(subtitle)
print(repr(subtitle))  # This will display the raw text with all special characters

The Holy Land. For thousands of years,
religious pilgrims of many faiths have come
to this sacred city to express their devotion to God. But just outside
the walls of Jerusalem lies the Valley of Hinnom, an ancient place
of unholy worship where evil deeds
were performed long ago in honor of God's enemy, Satan. In the book of Jeremiah,
God reports, through His prophet,
that people outside Jerusalem in a place called
the Valley of Hinnom, are sacrificing their sons,
and also their daughters. And God explicitly says,
"I never asked you to do this satanic ritual," and expresses that He's angry, that there's going to be
a terrible punishment for this. Historically speaking,
in the days of the Bible, not everybody was faithful
to the Lord. Satanism influenced
the Israelite community. And what they would do,
literally, was they'd create
a big pit of fire, and they actually
would sacrifice their firstborn male
through the fire. This was performed
in the Valley of Hinnom, which is actually the 

In [27]:
subtitle = subtitle.replace("\\", "")
sentences = sent_tokenize(subtitle)
#print(sentences)
organized_sent = {k:v for v,k in enumerate(sentences)}
tf_idf = TfidfVectorizer(min_df=2, 
                                    strip_accents='unicode',
                                    max_features=None,
                                    lowercase = True,
                                    token_pattern=r'w{1,}',
                                    ngram_range=(1, 3), 
                                    use_idf=True,
                                    smooth_idf=True,
                                    sublinear_tf=True,
                                    stop_words = 'english')

sentence_vectors = tf_idf.fit_transform(sentences)
sent_scores = np.array(sentence_vectors.sum(axis=1)).ravel()


In [19]:
N = 3
top_n_sentences = [sentences[index] for index in np.argsort(sent_scores, axis=0)[::-1][:N]]


In [20]:
# mapping the scored sentences with their indexes as in the subtitle
mapped_sentences = [(sentence,organized_sent[sentence]) for sentence in top_n_sentences]
# Ordering the top-n sentences in their original order
mapped_sentences = sorted(mapped_sentences, key = lambda x: x[1])
ordered_sentences = [element[0] for element in mapped_sentences]
# joining the ordered sentence
summary = " ".join(ordered_sentences)

In [21]:
print(summary)

They   found that after training the neural network 
you could give it all the hyperbolic geometry   invariants and like 90% of the time it will 
predict - it will guess the right signature. But if you already know sort of how the 
code works it saves you a lot of time. But we'll be able to do 
lots of things that we can't do right now.


In [8]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [28]:
# encode the subtitle using Bart tokenizer
input_tensor = tokenizer.encode( subtitle, return_tensors="pt", max_length=512)


In [29]:
outputs_tensor = model.generate(input_tensor, max_length=1024, min_length=120, length_penalty=2.0, num_beams=4, early_stopping=True)
#outputs_tensor

In [30]:
print(tokenizer.decode(outputs_tensor[0]))


</s><s>In the days of the Bible, not everybody was faithful to the Lord. Satanism influenced the Israelite community. People would sacrifice their firstborn male through a pit of fire. This was performed in the Valley of Hinnom, which is actually the Hebrew word today for "hell" Some people think of Satan as Beelzebub or Lucifer or "the evil one" or Old Nick. But is there really really a character of the Devil, as we know it, according to historian MIKKI BROCK? For more, go to CNN.com/soulmatestories.</s>


In [56]:
def summarize(link):
    unique_id = link.split("=")[-1]
    sub = YouTubeTranscriptApi.get_transcript(unique_id)
    subtitle = " ".join([x['text'] for x in sub])
    subtitle = subtitle.replace("\\", "")
    sentences = sent_tokenize(subtitle)
    organized_sent = {k:v for v,k in enumerate(sentences)}
    tf_idf = TfidfVectorizer(min_df=2, 
                                    strip_accents='unicode',
                                    max_features=None,
                                    lowercase = True,
                                    token_pattern=r'w{1,}',
                                    ngram_range=(1, 3), 
                                    use_idf=True,
                                    smooth_idf=True,
                                    sublinear_tf=True,
                                    stop_words = 'english')

    sentence_vectors = tf_idf.fit_transform(sentences)
    sent_scores = np.array(sentence_vectors.sum(axis=1)).ravel()
    N = 3
    top_n_sentences = [sentences[index] for index in np.argsort(sent_scores, axis=0)[::-1][:N]]
    mapped_sentences = [(sentence,organized_sent[sentence]) for sentence in top_n_sentences]
    mapped_sentences = sorted(mapped_sentences, key = lambda x: x[1])
    ordered_sentences = [element[0] for element in mapped_sentences]
    summary = " ".join(ordered_sentences)
    return summary
    